In [105]:
# imports
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import *
import time
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import requests
import os

In [106]:
# driver
path = "D:\Selenium\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.maximize_window()
website = 'https://www.purewow.com/fashion'
driver.get(website)

In [107]:
# dealing with popup
try:
    WebDriverWait(driver, 60).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//iframe[contains(@id, 'lightbox-iframe')]")))
except TimeoutException:
    print("TimeoutException: Popup not available in 60s, assume no popup")
except Exception as e:
    print(e)
    print("Error occured while waiting for popup")
else:
    skip_popup_button = driver.find_element_by_xpath(".//*[@id='layout']//button[text()='NO THANKS']")
    skip_popup_button.click()
    driver.switch_to.default_content()
    print("Popup closed successfully")
    time.sleep(3)

Popup closed successfully


In [108]:
# dataframe columns
blog_titles = []
blog_dates = []
author_names = []
blog_links = []
author_profile_links = []
thumbnail_links = []
thumbnail_credits = []

In [110]:
# button clicks
flag = 0
var = 2
temp = 0
while True:
    try:
        more_button = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='container show-more-container']//a[@href='javascript:void(0);']")))
    except TimeoutException:
        flag = 1
        print("TimeoutException: All clicks successful")
        break
    except Exception as e:
        print(e)
        print("Some error occured while clicking more_button")
        break
    else:
        more_button.click()
        temp += 1
        print("No TimeoutException: Single click successful")
        if temp == var:
            flag = 1
            print("No TimeoutException: All clicks successful")
            time.sleep(3)
            break

In [109]:
# function to extract data from cards
def extract_data_from_cards(*card_types):
    for card_type in card_types:
        for card in card_type:
            # blog title
            try:
                element = card.find_element_by_xpath(".//h1")
            except NoSuchElementException:
                try:
                    element = card.find_element_by_xpath(".//h4")
                except NoSuchElementException:
                    blog_title = ""
                else:
                    blog_title = element.text.strip()
            else:
                blog_title = element.text.strip()
            blog_titles.append(blog_title)

            # thumbnail credit
            try:
                element = card.find_element_by_xpath(".//span[@class='photo_credit']")
            except NoSuchElementException:
                thumbnail_credit = ""
            else:
                thumbnail_credit = element.text.strip()
            thumbnail_credits.append(thumbnail_credit)

            # blog link
            try:
                element = card.find_element_by_xpath(".//a")
            except NoSuchElementException:
                blog_link = ""
            else:
                element_href = str(element.get_attribute("href")).strip()
                if element_href[0] == '/':
                    blog_link = "https://www.purewow.com" + element_href
                else:
                    blog_link = element_href
            blog_links.append(blog_link)

            # thumbnail link
            try:
                element = card.find_element_by_xpath(".//img")
            except NoSuchElementException:
                thumbnail_link = ""
            else:
                thumbnail_link = str(element.get_attribute("src")).strip()
            thumbnail_links.append(thumbnail_link)

            # blank entries
            blog_dates.append("")
            author_names.append("")
            author_profile_links.append("")

In [111]:
# there are three different types of blog cards on webpage which are stored in cards_sliding, cards_big and cards_normal
if flag:
    print("Inside if")

    # cards_sliding
    try:
        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//div[@class='row slide-content']")))
    except TimeoutException:
        print("TimeoutException: No cards_sliding found")
    except Exception as e:
        print(e)
        print("Some error while waiting for cards_sliding")
    else:
        cards_sliding = driver.find_elements_by_xpath("//div[@class='row slide-content']")
        print(len(cards_sliding))

    # cards_big
    try:
        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//div[@class='col-sm-6 ']")))
    except TimeoutException:
        print("TimeoutException: No cards_big found")
    except Exception as e:
        print(e)
        print("Some error while waiting for cards_big")
    else:
        cards_big = driver.find_elements_by_xpath("//div[@class='col-sm-6 ']")
        print(len(cards_big))
    
    # cards_normal
    try:
        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//div[@class='col-sm-4' or @class='col-sm-4 ']")))
    except TimeoutException:
        print("TimeoutException: No cards_normal found")
    except Exception as e:
        print(e)
        print("Some error while waiting for cards_normal")
    else:
        cards_normal = driver.find_elements_by_xpath("//div[@class='col-sm-4' or @class='col-sm-4 ']")
        print(len(cards_normal))

    # extract data from cards
    extract_data_from_cards(cards_sliding, cards_big, cards_normal)

Inside if
6
4
9


In [112]:
# saving dataframe as csv
df = pd.DataFrame({"Blog Title": blog_titles, "Blog Date": blog_dates, "Author Name": author_names, "Thumbnail Credit": thumbnail_credits, "Blog Link": blog_links, "Author Profile Link": author_profile_links, "Thumbnail Link": thumbnail_links})
df
# csv_filename = os.path.basename(__file__).split('.')[0] + ".csv"
# csv_path = os.path.abspath(os.path.join(os.path.dirname(__file__), "../Data", csv_filename))
# df.to_csv(csv_path, index=False)
# print("df saved")

,Blog Title,Blog Date,Author Name,Thumbnail Credit,Blog Link,Author Profile Link,Thumbnail Link
0,,,,,https://www.purewow.com/fashion/stores-like-ma...,,https://purewows3.imgix.net/images/articles/20...
1,Sofía Vergara Takes Holiday Party-Chic to Anot...,,,STEVE GRANITZ/GETTY IMAGES,https://www.purewow.com/news/sofia-vergara-ski...,,https://purewows3.imgix.net/images/articles/20...
2,,,,,https://www.purewow.com/fashion/how-to-style-o...,,https://purewows3.imgix.net/images/articles/20...
3,,,,,https://www.purewow.com/news/rihanna-green-bra...,,https://purewows3.imgix.net/images/articles/20...
4,,,,,https://www.purewow.com/fashion/stores-like-ma...,,https://purewows3.imgix.net/images/articles/20...
5,,,,,https://www.purewow.com/news/sofia-vergara-ski...,,https://purewows3.imgix.net/images/articles/20...
6,The 32 Best Sales to Shop Right Now,,,MCKENZIE CORDELL,https://www.purewow.com/fashion/best-sales-to-...,,https://purewows3.imgix.net/images/articles/20...
7,"Telfar, Brandon Blackwood and 25 Black-Owned H...",,,SOFIA KRAUSHAAR,https://www.purewow.com/fashion/black-owned-ha...,,https://purewows3.imgix.net/images/articles/20...
8,The 5 Rules of Wearing Winter Boots in 2022,,,@GRACEATWOOD/INSTAGRAM,https://www.purewow.com/fashion/winter-boots-s...,,https://purewows3.imgix.net/images/articles/20...
9,"Mark Our Words, *This* Will Be the Top Fashion...",,,GETTY IMAGES,https://www.purewow.com/fashion/top-fashion-tr...,,https://purewows3.imgix.net/images/articles/20...


In [114]:
# driver.quit()